# [Day 16](https://adventofcode.com/2020/day/16): Ticket Translation

In [1]:
with open("../data/16.txt", "r") as f:
    lines = [l.strip() for l in f.readlines()]
    
ranges, myticket, tickets = lines[:20], lines[22], lines[25:]

## Part 1

In [2]:
import numpy as np
from itertools import chain
import re

def flatten(xss):
    return np.fromiter(chain.from_iterable(xss), dtype=int)

endpoints = flatten(re.findall(r"(\d+)-(\d+)", "".join(ranges)))
a, b = endpoints.reshape(-1, 2).T

nums = flatten(line.split(",") for line in tickets)
nums = nums.reshape(-1, 1)

invalid = ((nums < a) | (b < nums)).all(axis=1)
assert 25788 == np.sum(nums[invalid])

## Part 2

In [3]:
fields = np.zeros(20, dtype=int)
ticketno, rangeno = np.arange(20), np.arange(20)

valid = ~invalid.reshape(len(tickets), len(ranges)).any(axis=1)
nums = nums.reshape(len(tickets), len(ranges))
nums = nums.T[:, valid, np.newaxis]
inrange = (a <= nums) & (nums <= b)
inrange = inrange.reshape(*inrange.shape[:-1], len(nums), 2)  # Two ranges per field
inrange = inrange.any(axis=3).all(axis=1)

while len(inrange):
    i = np.sum(inrange, axis=1) == 1
    j = np.any(inrange[i], axis=0)
    fields[rangeno[j]] = ticketno[i]
    inrange  = inrange[~i][:, ~j]
    ticketno = ticketno[~i]
    rangeno = rangeno[~j]

myticket = np.array(myticket.split(","), dtype=int)
assert 3902565915559 == np.prod(myticket[fields[:6]])